![gearshift](img/LOGO_NAME.png)

# Gearshift

The gearshift tool is based in COMMISSION REGULATION (EU) 2017/1151 according to Amendment 6 to GTR No. 15 (Worldwide harmonized Light vehicles Test Procedures (WLTP) - Annex XXI sub-Annex 1 and 2. The aim of the Gearshift tool is obtain the Required Engine Speeds, the Available Powers, the Required Vehicle Speeds and the Gears for the whole WLTC based on the vehicle characteristics. The model should allow accurate calculation of final trace and the operating conditions of the engine.

## Restart Kernel

<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
    <b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Info</b><br>
    <p style="color: black">
        Restart kernel before execution
    </p>

![notebook](img/run_notebook.PNG)

</div>

##  Guidelines - How to execute <a class="anchor" id="execution"></a>

This section describes the generic execution approach. 

### Run with a GUI Interface <a class="anchor" id="gui"></a>

If you want to run it the gearshift tool using a GUI interface please follow the steps defined in this section.

#### Get demo input file <a class="anchor" id="demo"></a>

To download the demo file, select the Upload sheet and the Get Demo File section:

![demo](img/get_demo.PNG)

After execute Get Demo File, the template can be founded in the output folder (right side of the environment):

![download_demo](img/download_demo.PNG)


#### Select file for the execution <a class="anchor" id="file_exe"></a>

To upload file for the gearshift execution, select File Selection section and click in the upload button in the Upload sheet:

![upload_file](img/upload_file.PNG)

#### Data Preview <a class="anchor" id="prev_data"></a>

After uploading your data, you can do a preview of this selecting the sheet of the excel file in the Sheet Selection Preview section and clicking in the Preview button:

![preview](img/preview.PNG)

#### Run Gearshift Calculation tool <a class="anchor" id="run"></a>

After upload the input file you can execute the gearshift calculation tool selecting the Run sheet and clicking in the Run Gearshift button:  

![run](img/run.PNG)

The outputs of the gearshift can be founded in the output folder, you can download it by right clicking and selecting download or download the output.zip file that is generated automatically after execute gearshift tool:

![download](img/download.PNG)

### Terminal command

To run with terminal command go to section 1.3.2

## Execute

### GUI Execution interface <a class="anchor" id="gui_exe"></a>

In [2]:
import pandas as pd
import sys
import subprocess
from io import StringIO
import numpy as np
import ipywidgets as widgets
from tqdm import tqdm
import threading
import queue
import time
import zipfile
import os
from IPython.display import display





# Table definition
tab = widgets.Tab()
# Define the output terminal into application
out = widgets.Output(layout={'border': '1px solid black'})
out.overflow_x = 'scroll'
# Upload file
up = widgets.FileUpload(accept=".xlsx", multiple=False)
sheetNames = widgets.RadioButtons(
    options=['case', 'vehicle', 'engine', 'gearbox_ratios'],
    description='Sheet: ',
    disabled=False)

eraser = widgets.SelectMultiple(
    options=['tab','"'],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False)

rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

button_demo = widgets.Button(
    description='Get demo file',
    disabled=False,
    button_style='warning',
    tooltip='Click to Download demo file',
    icon='arrow-circle-down')

button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')

button_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='danger',
    tooltip='Click to Plot',
    icon='pencil')

graph_type = widgets.Dropdown(
    options=['Bar Chart', 'Line Chart'],
    value='Bar Chart',
    description='Chart Type:',
    disabled=False)

x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False)

y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Y-Axis:',
    disabled=False)

color_picker = widgets.ColorPicker(
    concise=False,
    description='Color Picker: ',
    value='lightblue',
    disabled=False)

toggle = widgets.ToggleButtons(
    options=['Preview  ', 'Info  ', 'Stats  '],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info', 'tachometer'])

button_run = widgets.Button(
    description='Run Gearshift',
    disabled=False,
    button_style='success',
    tooltip='Click to Plot',
    icon='seedling')

accordion = widgets.Accordion(children=[
    widgets.VBox([button_demo]),
    up, 
    widgets.VBox([sheetNames])])
accordion.set_title(0, 'Get Demo File')
accordion.set_title(1, 'File Selection')
accordion.set_title(2, 'Sheet Selection Preview')
accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_preview]),
    out
])

children = [
    accordion_box, 
    widgets.VBox([button_run, out]),
]

tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Run")
#tab.set_title(2, "Download")
tab

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

def content_reader():
    """
    This function reads and check the format of the input file
    
    :returns:
        - fileName (:py:class:`str`):
            
    """
    if up.value == {}:
        with out:
            print('No Excel file loaded')    
    else:
        fileName = next(iter(up.value))
        fileType = up.value[fileName]['metadata']['type']
        t = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
        
        if fileType == t:
            content = up.value[fileName]['content']
            with open(fileName, 'wb') as f: 
                f.write(content)
                
            return fileName

            
def df_converter():
    fileName = content_reader()
    if fileName is not None:
        input_data = pd.ExcelFile(fileName, engine="openpyxl")
        df = input_data.parse(sheetNames.value)
        numberCases =  input_data.parse("case")["case"].count()
        return df, numberCases - 1
    else:
        return None


def preview():
    df, numberCases = df_converter()
    with out:
        out.clear_output()
        print(f'\n The number of cases that will be run is: {numberCases} \n')
        print(f'\n -----The data in {sheetNames.value} sheet is:----- \n')
        if df is not None:
            display(df)
        else:
            print('Configuration is wrong/missing...')
            

def download():
    with out:
        out.clear_output()
        print('\n -----Downloading demo file:----- \n')
        
        command = !gearshift demo "./output"
        
        for line in command:
            print(line)

def run_command(my_queue):
    
    fileName = content_reader()
    command = !gearshift run "{fileName}" -O "./output"
    my_queue.put(command)
    


def runGS():
    
    df, numberCases = df_converter()
    
    with out:
        
        out.clear_output()
        
        my_queue = queue.Queue()
        
        commandGS = threading.Thread(target=run_command, args=(my_queue,))
        
        commandGS.start()
        print('\n -----Executing Gearshift Tool:----- \n')
        
        for i in tqdm(range(numberCases)):
            time.sleep(1.80)
        
        out.clear_output()
        
        command = my_queue.get()
        
        for line in command:
            print(line)
        
        print('\033[92m'+'\n***** Zip file have been created with the all files in the output folder *****\n'+'\033[0m')
        zipf = zipfile.ZipFile('output.zip', 'w', zipfile.ZIP_DEFLATED)
        zipdir('output/', zipf)
        zipf.close()
        
             
def preview_clicked(b):
    preview()
    
def download_demo(b):
    download()

def run_gearshift(b):
    runGS()


button_preview.on_click(preview_clicked)
button_demo.on_click(download_demo)
button_run.on_click(run_gearshift)


display(tab)

### Run with terminal command <a class="anchor" id="terminal"></a>

If you want execute the gearshift tool using the terminal command you can follow the instructions defined in the [documentation](https://gearshift-calculation-tool.readthedocs.io/en/latest/intro.html#quick-start) adding **!** at the beginning of the command:

```
!gearshift run "gs_input_demo.xlsx" -O "./output"
```

To upload the input file you can use the GUI interface to upload 1.2.1.2. Select file for the execution 